# Classification 2

## Exercise 1: Exploratory Data Analysis

### Overview

The objective of this course is to build models to predict customer churn for a fictitious telco company. Before we start creating models, let's begin by having a closer look at our data and doing some basic data wrangling.

Go through this notebook and modify the code accordingly (i.e. #TASK) based on the text and/or the comments.

### Data
Download data from here:
https://public.dhe.ibm.com/software/data/sw-library/cognos/mobile/C11/data/Telco_customer_churn.xlsx

Description of data (for a newer version)
https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

### Importing Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

# silence all warnings
warnings.filterwarnings("ignore")

# plotting settings
plt.style.use(['seaborn-paper'])
plt.rcParams['font.family'] = 'helvetica'

### Reading in the Data

In [2]:
# Use the parameter na_values=" " to convert any empty cells to a NA value.
# You may also need to use parameter engine='openpyxl') in newer versions of pandas if you encounter an XLRD error.
data = pd.read_excel("../data/raw/Telco_customer_churn.xlsx", na_values=" ", engine='openpyxl')
data.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.50,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,Yes,1,89,5340,Competitor had better devices


In [3]:
# Define columns to keep and filter the original dataset
cols_to_keep = ['CustomerID', 'Gender', 'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method', 'Monthly Charges', 'Total Charges', 'Churn Label']
data = data[cols_to_keep]

In [4]:
# TASK: Rename the multi-worded columns to remove the space 
# HINT: You can either manually remove the spaces in the column name list or use a loop to remove the space
data.columns = [('').join(col.split(' ')) for col in data.columns]

### 0. Basic Information

In [ ]:
# TASK: Display the number of rows and columns for the dataset
print(f"Rows & Columns: {data.shape}")

In [ ]:
# How many columns are numerical and how many are non-numerical
cat_cols = data.dtypes[data.dtypes == 'object'].index.tolist()
# include float, int
num_cols = data.dtypes[data.dtypes != 'object'].index.tolist()
# check that cat + num = total
assert(len(cat_cols) + len(num_cols) == len(data.columns))

# summary
print(f"Categorical columns: {cat_cols}\n")
print(f"Numerical columns: {num_cols}\n")
print(f"Total columns: {len(cat_cols)} categorical, {len(num_cols)} numerical.")

In [ ]:
# count the number of entries for each of the column. Are there any columns with missing values?
print(data.count())

# to check for missing % 
print("\n")
print(data.isnull().mean())     # TotalCharges has null values, but relatively small

In [ ]:
# count the number of unique values for each of the columns
print(data.nunique())

In [ ]:
# Based on this and the previous display, how would you describe the columns with a small number (less than 10) of unique values?
data.head(10)

We observe that columns with small number of unique values are mostly categorical columns where the field represents specific categories such as `DeviceProtection` covering responses like *'Yes', 'No', 'No internet servce'* etc. However, we do also observe categorical columns having a large number of unique values – `CustomerID`, which is expected as each customer id is uniquely identifying a specific customers hence, the column would soley be consisting of unique values. Otherwise, other columns of large number of unique values are all numerical columns where the column represents either continuous values such as `MonthyCharges` (in $) or discrete values like `TenureMonths` which represents the months the customer has been with the telco.

In [ ]:
# Let's analyze the values for the categorical features (columns with less than 10 unique values)
for id, row in data.nunique().iteritems(): # this counts the number of unique values for each feature and returns the result as a dictionary
    if(row < 10):
        print(f"{id} : {data[id].unique()}")

In [ ]:
# For columns with 3 or 4 unique values, display them to see if they make sense
for col in ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', "PaymentMethod"]:
    print(f"{col} : {data[col].unique()}")

**Observations**

- The value 'No phone service' found in MultipleLines is already captured by the PhoneService feature ('No' value)
- The value 'No internet service' found in the several features is already captured by InternetService feature ('No' value)
- Values that are longer or more complex may need to be simplified.

Conclusion: These values can be considered duplicated information as they are found in the PhoneService and InternetService features. There are several options to consider here:

- Retain all features and values as is
- Convert the 'No Internet Service'/'No phone service' to 'No' in the features as PhoneService and InternetService features has already captured this information
- Remove the PhoneService feature as MultipleLines feature has this information. To remove the InternetService feature, we would have to 'fold in' the values in the other features e.g. the values for OnlineSecurity could be changed to ['DSL_No','DSL_Yes','FiberOptic_No','FiberOptic_Yes','No internet service']

For this course, we will be using the second option (without justification). You are encouraged to test the others options during modelling to see if there are any impact.

### Data Wrangling

Based on the discoveries made above, we will be modifying our data before continuing the exploration.

In [ ]:
# Replace 'No phone service'
data['MultipleLines'] = data['MultipleLines'].replace({'No phone service':'No'})

In [ ]:
# similar to the operation for 'No phone service' above
for col in ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']:
    data[col] = data[col].replace({'No internet service': 'No'})

In [ ]:
# Simplify the values made up of phrases
data['PaymentMethod'] = data['PaymentMethod'].replace({
    'Bank transfer (automatic)':'transfer',
    'Credit card (automatic)':'creditcard',
    'Electronic check':'echeck',
    'Mailed check':'mcheck'
})

data['InternetService'] = data['InternetService'].replace({
    'Fiber optic':'FiberOptic'
})

data['Contract'] = data['Contract'].replace({
    'Month-to-month':'M2M',
    'One year':'OneYear',
    'Two year':'TwoYear'
})

In [ ]:
# Remove the rows with empty TotalCharges value
data = data[data["TotalCharges"].notnull()]

In [ ]:
# After data wrangling, repeat prints
print("Rows & Columns: {}".format(data.shape))
print("################################################")
# Number of unique values for each of the columns
print(data.nunique())
print("################################################")
# Check the data types
print(data.dtypes)
print("################################################")
# Display first few values
print(data.head())

In [ ]:
# Randomly display 1 row from the dataframe
print(data.sample(n=1).iloc[0])

In [ ]:
# TASK: Save the data as a CSV fiile
data.to_csv("../data/interim/telco_churn.csv", index=False)

### 1. Additional Exploration

**TASK:** This is the open-ended section of the exercise. Use any exploration techniques that you know to further explore and understand your data. We expect a number of visualizations that can show the relationships between features as well as between features and the outcome variable 'ChurnLabel'. Some of the questions in the quiz may require you to perform additional analyses.

Based on the previous section of data exploration, we have already covered the following questions that we might have about the dataset:
>>
1. What is the shape of your data? Number of rows and columns.
2. How many of the columns are numerical and how many are categorical?
3. What is the name of the column to be predicted?
4. For the numerical columns, how many missing values are there for each column?
5. For the categorical columns, how many misisng values are there for each column?

Those are some questions that we can simply obtain answer to by looking at the dataframe through its various methods. However, we still have some concerns about the dataset that we want to explore through effective visualisations. Hence, in the following section, we want to explore the following questions:
>>
1. For the numerical columns, what does the distributions look like?
2. How are the various attributes correlated to the outcome variable?
3. What visualizations can you use to highlight outliers in the data?

##### Recapping on the columns

Before we begin to decide on the type of visualisations to use to aim our understanding, let's recap on the columns we have and the data type of it.

In [ ]:
print(f"Categorical columns: {cat_cols}\n")
print(f"Numerical columns: {num_cols}\n")
print(f"Total columns: {len(cat_cols)} categorical, {len(num_cols)} numerical.")

##### Utilities Functions
Let's define some helper functions that will help us iterate through all the columns with same datatype. 

In [ ]:
# Example: Look at Churn vs MonthCharges
plt.clf()
for label in ['Yes','No']:
    subset = data[data.ChurnLabel==label]
    
    # Draw the density plot
    sns.distplot(subset['MonthlyCharges'], hist = False, kde = True,
                 kde_kws = {'linewidth': 3, 'shade':True},
                 label = label)
    
# Plot formatting
plt.legend(prop={'size': 16}, title = 'ChurnLabel')
plt.title('Density Plot with ChurnLabel')
plt.xlabel('') # Monthly Charges
plt.ylabel('Density')
plt.show()

In [ ]:
# Additional Exploration